# 📊 Model Performance Analysis & EDA
## Interactive Exploration of Model Test Outputs

This notebook provides comprehensive analysis of model predictions and performance metrics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("📦 Libraries loaded successfully!")

## 1. Load Available Models

In [ ]:
def get_available_models(predictions_dir="data/predictions"):
    """Get list of available models"""
    predictions_dir = Path(predictions_dir)
    
    if not predictions_dir.exists():
        return []
    
    models = []
    for model_dir in predictions_dir.iterdir():
        if model_dir.is_dir():
            # Check if there are prediction files
            pred_files = list(model_dir.glob("*_test_preds_*.csv"))
            if pred_files:
                models.append(model_dir.name)
    
    return sorted(models)

# Get available models
available_models = get_available_models()
print(f"📋 Available Models ({len(available_models)}):")
for i, model in enumerate(available_models, 1):
    print(f"  {i}. {model}")

## 2. Select and Load Model Data

In [ ]:
# Select model to analyze
MODEL_NAME = "mlp"  # Change this to analyze different models

def load_model_predictions(model_name, predictions_dir="data/predictions"):
    """Load model predictions and return test and train dataframes"""
    model_dir = Path(predictions_dir) / model_name
    
    # Get latest files
    test_pred_files = list(model_dir.glob("*_test_preds_*.csv"))
    train_pred_files = list(model_dir.glob("*_train_preds_*.csv"))
    
    if not test_pred_files:
        print(f"❌ No test prediction files found for {model_name}")
        return None, None
    
    # Use latest files
    latest_test = max(test_pred_files, key=lambda x: x.stat().st_mtime)
    latest_train = max(train_pred_files, key=lambda x: x.stat().st_mtime) if train_pred_files else None
    
    print(f"📂 Loading data for {model_name}:")
    print(f"  Test predictions: {latest_test.name}")
    if latest_train:
        print(f"  Train predictions: {latest_train.name}")
    
    # Load data
    test_df = pd.read_csv(latest_test)
    train_df = pd.read_csv(latest_train) if latest_train else None
    
    # Convert timestamps
    if 'timestamp' in test_df.columns:
        test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
    if train_df is not None and 'timestamp' in train_df.columns:
        train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
    
    return test_df, train_df

# Load the selected model
test_df, train_df = load_model_predictions(MODEL_NAME)

if test_df is not None:
    print(f"\n✅ Data loaded successfully!")
    print(f"📊 Test set shape: {test_df.shape}")
    if train_df is not None:
        print(f"📊 Train set shape: {train_df.shape}")
else:
    print(f"❌ Failed to load data for {MODEL_NAME}")

## 3. Data Overview & Summary Statistics

## 2.1. System Architecture & Model Information

This section provides information about the current system architecture, champion model status, data encoding, and model-specific details.

In [ ]:
import yaml

def get_champion_model_info():
    """Get current champion model information"""
    champion_file = Path("models/champion.txt")
    if champion_file.exists():
        with open(champion_file, 'r') as f:
            champion_model = f.read().strip()
        return champion_model
    return "Unknown"

def load_model_metadata(model_name, predictions_dir="data/predictions"):
    """Load model metadata from YAML file"""
    model_dir = Path(predictions_dir) / model_name
    metadata_files = list(model_dir.glob("*_metadata_*.yaml"))
    
    if not metadata_files:
        return None
    
    latest_metadata = max(metadata_files, key=lambda x: x.stat().st_mtime)
    
    try:
        with open(latest_metadata, 'r') as f:
            content = f.read()
            # Filter out problematic NumPy serializations
            lines = content.split('\n')
            filtered_lines = []
            skip_lines = False
            
            for line in lines:
                if 'test_metrics:' in line or 'train_metrics:' in line:
                    skip_lines = True
                    continue
                if skip_lines and (line.startswith('  ') or line.strip() == ''):
                    continue
                if skip_lines and not line.startswith(' '):
                    skip_lines = False
                
                if not skip_lines:
                    filtered_lines.append(line)
            
            filtered_content = '\n'.join(filtered_lines)
            metadata = yaml.safe_load(filtered_content)
            
            # Ensure we have the model name
            if metadata and 'model' not in metadata:
                metadata['model'] = model_name
        return metadata
    except Exception as e:
        print(f"Warning: Could not load metadata: {e}")
        return {'model': model_name}

def get_system_architecture_info(model_name, metadata):
    """Get system architecture and data encoding information"""
    info = {}
    
    # Champion model status
    champion_model = get_champion_model_info()
    info['champion_model'] = champion_model
    info['is_champion'] = model_name == champion_model
    
    # Data encoding status
    if metadata and 'data_type' in metadata:
        data_type = metadata['data_type']
        info['data_type'] = data_type
        if data_type == 'dense':
            info['encoding_status'] = "✅ Using autoencoder embeddings (64-dimensional dense representation)"
        else:
            info['encoding_status'] = "📊 Using scaled wide format (direct feature encoding)"
    else:
        info['encoding_status'] = "❓ Data encoding status unknown"
    
    # MLP-specific architecture information
    if model_name == 'mlp' and metadata and 'best_params' in metadata:
        best_params = metadata['best_params']
        info['mlp_architecture'] = {
            'hidden_layers': best_params.get('hidden_layer_sizes', 'Unknown'),
            'alpha': best_params.get('alpha', 'Unknown'),
            'learning_rate_init': best_params.get('learning_rate_init', 'Unknown'),
            'max_iter': best_params.get('max_iter', 'Unknown')
        }
    
    return info

def display_system_information(model_name, metadata):
    """Display system architecture and model information"""
    print(f"{'='*60}")
    print("🏗️  SYSTEM ARCHITECTURE & MODEL INFORMATION")
    print(f"{'='*60}")
    
    # Get system info
    sys_info = get_system_architecture_info(model_name, metadata)
    
    # Champion model status
    champion_status = "🏆 CHAMPION MODEL" if sys_info.get('is_champion') else "📈 Non-champion model"
    print(f"\nModel Status: {champion_status}")
    print(f"Current Champion: {sys_info.get('champion_model', 'Unknown')}")
    
    # Data encoding information
    print(f"\nData Architecture:")
    print(f"  {sys_info.get('encoding_status', 'Unknown')}")
    
    if 'data_type' in sys_info:
        if sys_info['data_type'] == 'dense':
            print(f"  • Features preprocessed through autoencoder neural network")
            print(f"  • 64-dimensional compressed representation")
            print(f"  • Better for capturing complex feature interactions")
        else:
            print(f"  • Direct feature scaling and engineering")
            print(f"  • Wide format with original feature structure")
            print(f"  • Better for interpretability and linear relationships")
    
    # MLP-specific information
    if model_name == 'mlp' and 'mlp_architecture' in sys_info:
        arch = sys_info['mlp_architecture']
        print(f"\n🧠 MLP Neural Network Architecture:")
        print(f"  • Hidden Layers: {arch['hidden_layers']} neurons")
        print(f"  • Regularization (alpha): {arch['alpha']:.6f}")
        print(f"  • Learning Rate: {arch['learning_rate_init']:.6f}")
        print(f"  • Max Iterations: {arch['max_iter']}")
        print(f"  • Architecture: Input → {arch['hidden_layers'][0]} → {arch['hidden_layers'][1]} → Output")

# Load metadata and display system information
if test_df is not None:
    metadata = load_model_metadata(MODEL_NAME)
    display_system_information(MODEL_NAME, metadata)

## 2.5. Time Range Selection (Optional)

Select a specific time range for detailed analysis. This is useful for focusing on specific periods of interest.

In [ ]:
def get_time_range_options(test_df):
    """Get available time range options"""
    if 'timestamp' not in test_df.columns:
        return {}
    
    test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
    start_date = test_df['timestamp'].min()
    end_date = test_df['timestamp'].max()
    full_duration = end_date - start_date
    
    options = {}
    
    # Always include full range
    options['full'] = ('Full Range', start_date, end_date)
    
    # Add recent periods
    if full_duration.days >= 30:
        options['last_30'] = ('Last 30 Days', 
                            max(start_date, end_date - pd.Timedelta(days=30)), end_date)
    if full_duration.days >= 14:
        options['last_14'] = ('Last 14 Days', 
                            max(start_date, end_date - pd.Timedelta(days=14)), end_date)
    if full_duration.days >= 7:
        options['last_7'] = ('Last 7 Days', 
                           max(start_date, end_date - pd.Timedelta(days=7)), end_date)
    
    # Add quarterly options if data spans multiple years
    if full_duration.days > 365:
        current_year = end_date.year
        options['q4'] = (f'Q4 {current_year}', 
                       max(start_date, pd.Timestamp(f'{current_year}-10-01')), end_date)
        options['q3'] = (f'Q3 {current_year}', 
                       max(start_date, pd.Timestamp(f'{current_year}-07-01')), 
                       min(end_date, pd.Timestamp(f'{current_year}-09-30')))
    
    # Filter out options that would be too small
    valid_options = {}
    for key, (label, opt_start, opt_end) in options.items():
        if (opt_end - opt_start).days >= 7:  # At least 7 days
            valid_options[key] = (label, opt_start, opt_end)
    
    return valid_options

def select_time_range_interactive(test_df):
    """Interactive time range selection for Jupyter"""
    options = get_time_range_options(test_df)
    
    if not options:
        print("⚠️ No timestamp data available for time range selection")
        return None
    
    print("📅 Available Time Ranges:")
    for i, (key, (label, start, end)) in enumerate(options.items(), 1):
        duration = end - start
        print(f"  {i}. {label}: {start.date()} to {end.date()} ({duration.days} days)")
    
    print("\n💡 You can modify the SELECTED_TIME_RANGE variable below to choose a specific range")
    return options

# Set up time range selection
if test_df is not None and 'timestamp' in test_df.columns:
    time_options = select_time_range_interactive(test_df)
    
    # You can change this to select different time ranges:
    # Options: 'full', 'last_30', 'last_14', 'last_7', 'q4', 'q3'
    SELECTED_TIME_RANGE = 'full'  # Change this to focus on specific periods
    
    if SELECTED_TIME_RANGE in time_options:
        selected_range = time_options[SELECTED_TIME_RANGE]
        TIME_RANGE_LABEL, TIME_START, TIME_END = selected_range
        
        # Filter data for visualization (keep original for metrics)
        viz_test_df = test_df[(test_df['timestamp'] >= TIME_START) & (test_df['timestamp'] <= TIME_END)].copy()
        print(f"\n✅ Selected: {TIME_RANGE_LABEL}")
        print(f"📊 Filtered data: {len(viz_test_df)} samples ({len(viz_test_df)/len(test_df)*100:.1f}% of total)")
    else:
        viz_test_df = test_df.copy()
        TIME_RANGE_LABEL = "Full Range"
        print(f"\n✅ Using full range for analysis")
else:
    viz_test_df = test_df.copy() if test_df is not None else None
    TIME_RANGE_LABEL = "Full Range"

In [ ]:
if test_df is not None:
    print(f"🔍 DATA OVERVIEW FOR {MODEL_NAME.upper()}")
    print("=" * 50)
    
    # Basic info
    print(f"\n📋 Columns: {list(test_df.columns)}")
    print(f"📏 Test set rows: {len(test_df):,}")
    
    if 'timestamp' in test_df.columns:
        print(f"📅 Date range: {test_df['timestamp'].min()} to {test_df['timestamp'].max()}")
        print(f"⏱️  Duration: {test_df['timestamp'].max() - test_df['timestamp'].min()}")
    
    # Display first few rows
    print(f"\n📑 First 5 rows:")
    display(test_df.head())
    
    # Summary statistics
    print(f"\n📊 Summary Statistics:")
    display(test_df[['true_count', 'pred_count']].describe())

## 4. Performance Metrics

In [ ]:
def calculate_comprehensive_metrics(df, set_name="Test"):
    """Calculate comprehensive performance metrics"""
    if df is None or 'true_count' not in df.columns or 'pred_count' not in df.columns:
        return {}
    
    y_true = df['true_count']
    y_pred = df['pred_count']
    
    # Basic metrics
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    
    # Percentage errors
    non_zero_mask = y_true != 0
    if non_zero_mask.sum() > 0:
        mape = np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
    else:
        mape = np.inf
    
    # R-squared
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
    
    # Additional metrics
    median_ae = np.median(np.abs(y_true - y_pred))
    max_error = np.max(np.abs(y_true - y_pred))
    
    metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE (%)': mape,
        'R²': r2,
        'Median AE': median_ae,
        'Max Error': max_error
    }
    
    return metrics

if test_df is not None:
    # Calculate metrics for test set
    test_metrics = calculate_comprehensive_metrics(test_df, "Test")
    
    print(f"📈 PERFORMANCE METRICS FOR {MODEL_NAME.upper()}")
    print("=" * 50)
    
    print(f"\n🎯 Test Set Performance:")
    for metric, value in test_metrics.items():
        print(f"  {metric:>12}: {value:10.4f}")
    
    # Calculate metrics for train set if available
    if train_df is not None:
        train_metrics = calculate_comprehensive_metrics(train_df, "Train")
        print(f"\n🏋️ Train Set Performance:")
        for metric, value in train_metrics.items():
            print(f"  {metric:>12}: {value:10.4f}")
        
        # Compare train vs test
        print(f"\n📊 Train vs Test Comparison:")
        print(f"  {'Metric':>12} | {'Train':>10} | {'Test':>10} | {'Difference':>12}")
        print("-" * 50)
        for metric in ['MAE', 'RMSE', 'R²']:
            if metric in train_metrics and metric in test_metrics:
                diff = test_metrics[metric] - train_metrics[metric]
                print(f"  {metric:>12} | {train_metrics[metric]:>10.4f} | {test_metrics[metric]:>10.4f} | {diff:>+12.4f}")

## 5. Time Series Visualization

In [ ]:
if test_df is not None:
    # Use viz_test_df (filtered data) for visualizations, but keep test_df for metrics
    if viz_test_df is not None:
        # Create comprehensive time series plots
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle(f'Time Series Analysis: {MODEL_NAME.upper()} ({TIME_RANGE_LABEL})', fontsize=16, fontweight='bold')
        
        # Calculate key metrics for analysis text
        y_true = viz_test_df['true_count']
        y_pred = viz_test_df['pred_count']
        residuals = y_true - y_pred
        mae = np.mean(np.abs(residuals))
        rmse = np.sqrt(np.mean(residuals ** 2))
        correlation = np.corrcoef(y_true, y_pred)[0, 1]
        
        # 1. Full time series
        ax1 = axes[0, 0]
        if 'timestamp' in viz_test_df.columns:
            x_data = viz_test_df['timestamp']
            xlabel = 'Date'
        else:
            x_data = range(len(viz_test_df))
            xlabel = 'Index'
        
        ax1.plot(x_data, viz_test_df['true_count'], label='Actual', color='blue', linewidth=1.2, alpha=0.8)
        ax1.plot(x_data, viz_test_df['pred_count'], label='Predicted', color='red', linewidth=1.2, alpha=0.8, linestyle='--')
        ax1.set_title('Time Series: Predictions vs Actual', fontsize=12, fontweight='bold')
        ax1.set_xlabel(xlabel)
        ax1.set_ylabel('Count')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Add analysis text
        time_text = f"""Analysis: The model shows {'strong' if correlation > 0.8 else 'moderate' if correlation > 0.6 else 'weak'} temporal correlation (r={correlation:.3f}).
        MAE={mae:.1f}, RMSE={rmse:.1f}. The predictions {'closely follow' if mae < y_true.mean() * 0.1 else 'generally track' if mae < y_true.mean() * 0.2 else 'loosely follow'} the actual patterns.
        {'Seasonal patterns are well captured.' if correlation > 0.7 else 'Some temporal patterns may be missed.'}"""
        ax1.text(0.02, 0.98, time_text, transform=ax1.transAxes, verticalalignment='top', 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7), fontsize=8)
        
        # 2. Predicted vs Actual Scatter
        ax2 = axes[0, 1]
        ax2.scatter(viz_test_df['true_count'], viz_test_df['pred_count'], alpha=0.6, color='blue', s=15)
        
        # Perfect prediction line
        min_val = min(viz_test_df['true_count'].min(), viz_test_df['pred_count'].min())
        max_val = max(viz_test_df['true_count'].max(), viz_test_df['pred_count'].max())
        ax2.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction', linewidth=2)
        
        ax2.set_title('Predicted vs Actual Values', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Actual Count')
        ax2.set_ylabel('Predicted Count')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Add analysis text
        scatter_text = f"""Analysis: Points {'cluster tightly' if rmse < y_true.std() * 0.5 else 'spread moderately' if rmse < y_true.std() else 'spread widely'} around perfect line.
        R²={correlation**2:.3f} indicates {correlation**2*100:.1f}% variance explained.
        {'Excellent prediction accuracy' if correlation > 0.9 else 'Good prediction accuracy' if correlation > 0.8 else 'Moderate prediction accuracy' if correlation > 0.6 else 'Room for improvement'} for this range."""
        ax2.text(0.02, 0.98, scatter_text, transform=ax2.transAxes, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7), fontsize=8)
        
        # 3. Residuals distribution
        ax3 = axes[1, 0]
        ax3.hist(residuals, bins=50, alpha=0.7, color='skyblue', edgecolor='black', density=True)
        ax3.axvline(0, color='red', linestyle='--', label='Zero Error', linewidth=2)
        ax3.axvline(residuals.mean(), color='orange', linestyle='-', label=f'Mean Error ({residuals.mean():.1f})', linewidth=2)
        ax3.set_title('Residuals Distribution', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Residual (Actual - Predicted)')
        ax3.set_ylabel('Density')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Add analysis text
        skewness = residuals.skew()
        bias_text = f"""Analysis: Mean error = {residuals.mean():.2f} ({'systematic bias' if abs(residuals.mean()) > rmse * 0.1 else 'minimal bias'}).
        Distribution is {'highly skewed' if abs(skewness) > 1 else 'moderately skewed' if abs(skewness) > 0.5 else 'approximately normal'} (skew={skewness:.2f}).
        {'Model tends to over-predict' if residuals.mean() < -5 else 'Model tends to under-predict' if residuals.mean() > 5 else 'Model is well-calibrated'}."""
        ax3.text(0.02, 0.98, bias_text, transform=ax3.transAxes, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.7), fontsize=8)
        
        # 4. Absolute error over time
        ax4 = axes[1, 1]
        abs_errors = np.abs(residuals)
        ax4.plot(x_data, abs_errors, color='orange', alpha=0.7, linewidth=1.2)
        ax4.axhline(mae, color='red', linestyle='--', label=f'Mean Absolute Error ({mae:.1f})', linewidth=2)
        ax4.set_title('Absolute Error Over Time', fontsize=12, fontweight='bold')
        ax4.set_xlabel(xlabel)
        ax4.set_ylabel('Absolute Error')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # Add analysis text
        error_stability = abs_errors.std()
        error_text = f"""Analysis: Error consistency - std={error_stability:.2f} ({'very stable' if error_stability < mae * 0.5 else 'stable' if error_stability < mae else 'variable'}).
        Max error = {abs_errors.max():.1f}, occurs {'frequently' if (abs_errors > mae * 2).mean() > 0.1 else 'occasionally'}.
        {'Error patterns suggest model limitations' if abs_errors.max() > mae * 5 else 'Error patterns are reasonable'}."""
        ax4.text(0.02, 0.98, error_text, transform=ax4.transAxes, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.7), fontsize=8)
        
        plt.tight_layout()
        plt.show()
        
        # Save the plot
        time_suffix = f"_{TIME_START.strftime('%Y%m%d')}_{TIME_END.strftime('%Y%m%d')}" if 'TIME_START' in globals() and TIME_RANGE_LABEL != "Full Range" else ""
        output_file = f"{MODEL_NAME}_enhanced_analysis{time_suffix}.png"
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"\n📊 Enhanced visualization saved as: {output_file}")
        
        print(f"📊 Enhanced time series analysis completed for {MODEL_NAME} ({TIME_RANGE_LABEL})")
    else:
        print("❌ No data available for visualization")

## 6. Distribution Analysis

In [ ]:
if viz_test_df is not None:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Distribution Analysis: {MODEL_NAME.upper()} ({TIME_RANGE_LABEL})', fontsize=16, fontweight='bold')
    
    # Calculate residuals for filtered data
    residuals = viz_test_df['true_count'] - viz_test_df['pred_count']
    
    # 1. Actual vs Predicted distributions
    ax1 = axes[0, 0]
    ax1.hist(viz_test_df['true_count'], bins=50, alpha=0.7, label='Actual', color='blue', density=True)
    ax1.hist(viz_test_df['pred_count'], bins=50, alpha=0.7, label='Predicted', color='red', density=True)
    ax1.set_title('Distribution: Actual vs Predicted')
    ax1.set_xlabel('Count')
    ax1.set_ylabel('Density')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Residuals distribution
    ax2 = axes[0, 1]
    ax2.hist(residuals, bins=50, alpha=0.7, color='orange', edgecolor='black')
    ax2.axvline(0, color='red', linestyle='--', linewidth=2)
    ax2.axvline(np.mean(residuals), color='green', linestyle=':', linewidth=2, label=f'Mean: {np.mean(residuals):.2f}')
    ax2.set_title('Residuals Distribution')
    ax2.set_xlabel('Residual (Actual - Predicted)')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Q-Q plot for residuals
    ax3 = axes[1, 0]
    from scipy import stats
    stats.probplot(residuals, dist="norm", plot=ax3)
    ax3.set_title('Q-Q Plot: Residuals vs Normal Distribution')
    ax3.grid(True, alpha=0.3)
    
    # 4. Absolute error distribution
    ax4 = axes[1, 1]
    abs_errors = np.abs(residuals)
    mae_filtered = np.mean(abs_errors)
    median_ae_filtered = np.median(abs_errors)
    ax4.hist(abs_errors, bins=50, alpha=0.7, color='purple', edgecolor='black')
    ax4.axvline(mae_filtered, color='red', linestyle='--', linewidth=2, label=f'MAE: {mae_filtered:.2f}')
    ax4.axvline(median_ae_filtered, color='green', linestyle=':', linewidth=2, label=f'Median AE: {median_ae_filtered:.2f}')
    ax4.set_title('Absolute Error Distribution')
    ax4.set_xlabel('Absolute Error')
    ax4.set_ylabel('Frequency')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Distribution analysis completed for {MODEL_NAME} ({TIME_RANGE_LABEL})")
else:
    print("❌ No data available for distribution analysis")

## 7. Scatter Plot Analysis

In [ ]:
if test_df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f'Scatter Plot Analysis: {MODEL_NAME.upper()}', fontsize=16, fontweight='bold')
    
    # 1. Predicted vs Actual
    ax1 = axes[0]
    ax1.scatter(test_df['true_count'], test_df['pred_count'], alpha=0.6, color='blue', s=20)
    
    # Perfect prediction line
    min_val = min(test_df['true_count'].min(), test_df['pred_count'].min())
    max_val = max(test_df['true_count'].max(), test_df['pred_count'].max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    # Add trend line
    z = np.polyfit(test_df['true_count'], test_df['pred_count'], 1)
    p = np.poly1d(z)
    ax1.plot(test_df['true_count'], p(test_df['true_count']), 'g-', alpha=0.8, linewidth=2, label=f'Trend: y={z[0]:.3f}x+{z[1]:.1f}')
    
    ax1.set_title('Predicted vs Actual')
    ax1.set_xlabel('Actual Count')
    ax1.set_ylabel('Predicted Count')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add R² annotation
    ax1.text(0.05, 0.95, f'R² = {test_metrics["R²"]:.4f}', transform=ax1.transAxes, 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8), fontsize=12)
    
    # 2. Residuals vs Predicted
    ax2 = axes[1]
    residuals = test_df['true_count'] - test_df['pred_count']
    ax2.scatter(test_df['pred_count'], residuals, alpha=0.6, color='orange', s=20)
    ax2.axhline(0, color='red', linestyle='--', linewidth=2)
    
    # Add trend line to check for bias
    z_res = np.polyfit(test_df['pred_count'], residuals, 1)
    p_res = np.poly1d(z_res)
    ax2.plot(test_df['pred_count'], p_res(test_df['pred_count']), 'g-', alpha=0.8, linewidth=2, 
             label=f'Trend: slope={z_res[0]:.6f}')
    
    ax2.set_title('Residuals vs Predicted')
    ax2.set_xlabel('Predicted Count')
    ax2.set_ylabel('Residuals')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Scatter plot analysis completed for {MODEL_NAME}")

## 8. Error Analysis by Time Patterns

In [ ]:
if test_df is not None and 'timestamp' in test_df.columns:
    # Add time components
    test_df['hour'] = test_df['timestamp'].dt.hour
    test_df['day_of_week'] = test_df['timestamp'].dt.dayofweek
    test_df['month'] = test_df['timestamp'].dt.month
    test_df['abs_error'] = np.abs(test_df['true_count'] - test_df['pred_count'])
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Error Analysis by Time Patterns: {MODEL_NAME.upper()}', fontsize=16, fontweight='bold')
    
    # 1. Error by hour of day
    ax1 = axes[0, 0]
    hourly_error = test_df.groupby('hour')['abs_error'].agg(['mean', 'std']).reset_index()
    ax1.bar(hourly_error['hour'], hourly_error['mean'], alpha=0.7, color='skyblue', edgecolor='black')
    ax1.errorbar(hourly_error['hour'], hourly_error['mean'], yerr=hourly_error['std'], 
                 fmt='none', color='red', capsize=3)
    ax1.set_title('Average Absolute Error by Hour of Day')
    ax1.set_xlabel('Hour of Day')
    ax1.set_ylabel('Mean Absolute Error')
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(range(0, 24, 2))
    
    # 2. Error by day of week
    ax2 = axes[0, 1]
    daily_error = test_df.groupby('day_of_week')['abs_error'].agg(['mean', 'std']).reset_index()
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    ax2.bar(range(7), daily_error['mean'], alpha=0.7, color='lightgreen', edgecolor='black')
    ax2.errorbar(range(7), daily_error['mean'], yerr=daily_error['std'], 
                 fmt='none', color='red', capsize=3)
    ax2.set_title('Average Absolute Error by Day of Week')
    ax2.set_xlabel('Day of Week')
    ax2.set_ylabel('Mean Absolute Error')
    ax2.set_xticks(range(7))
    ax2.set_xticklabels(days)
    ax2.grid(True, alpha=0.3)
    
    # 3. Error by month
    ax3 = axes[1, 0]
    monthly_error = test_df.groupby('month')['abs_error'].agg(['mean', 'std']).reset_index()
    ax3.bar(monthly_error['month'], monthly_error['mean'], alpha=0.7, color='coral', edgecolor='black')
    ax3.errorbar(monthly_error['month'], monthly_error['mean'], yerr=monthly_error['std'], 
                 fmt='none', color='red', capsize=3)
    ax3.set_title('Average Absolute Error by Month')
    ax3.set_xlabel('Month')
    ax3.set_ylabel('Mean Absolute Error')
    ax3.grid(True, alpha=0.3)
    ax3.set_xticks(range(1, 13))
    
    # 4. Correlation heatmap
    ax4 = axes[1, 1]
    correlation_data = test_df[['true_count', 'pred_count', 'abs_error', 'hour', 'day_of_week', 'month']]
    correlation_matrix = correlation_data.corr()
    
    im = ax4.imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    ax4.set_xticks(range(len(correlation_matrix.columns)))
    ax4.set_yticks(range(len(correlation_matrix.columns)))
    ax4.set_xticklabels(correlation_matrix.columns, rotation=45)
    ax4.set_yticklabels(correlation_matrix.columns)
    ax4.set_title('Correlation Matrix')
    
    # Add correlation values
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            ax4.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                    ha="center", va="center", color="black" if abs(correlation_matrix.iloc[i, j]) < 0.5 else "white")
    
    plt.colorbar(im, ax=ax4)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Time pattern analysis completed for {MODEL_NAME}")
    
    # Print insights
    print(f"\n🔍 TIME PATTERN INSIGHTS:")
    worst_hour = hourly_error.loc[hourly_error['mean'].idxmax(), 'hour']
    best_hour = hourly_error.loc[hourly_error['mean'].idxmin(), 'hour']
    print(f"  Worst performing hour: {worst_hour:02d}:00 (MAE: {hourly_error.loc[hourly_error['hour']==worst_hour, 'mean'].iloc[0]:.2f})")
    print(f"  Best performing hour:  {best_hour:02d}:00 (MAE: {hourly_error.loc[hourly_error['hour']==best_hour, 'mean'].iloc[0]:.2f})")
    
    worst_day = daily_error.loc[daily_error['mean'].idxmax(), 'day_of_week']
    best_day = daily_error.loc[daily_error['mean'].idxmin(), 'day_of_week']
    print(f"  Worst performing day: {days[worst_day]} (MAE: {daily_error.loc[daily_error['day_of_week']==worst_day, 'mean'].iloc[0]:.2f})")
    print(f"  Best performing day:  {days[best_day]} (MAE: {daily_error.loc[daily_error['day_of_week']==best_day, 'mean'].iloc[0]:.2f})")

## 9. Model Comparison Summary

In [ ]:
# Load and compare all available models
def compare_all_models():
    """Compare performance of all available models"""
    comparison_results = []
    
    for model in available_models:
        try:
            test_df_model, _ = load_model_predictions(model)
            if test_df_model is not None:
                metrics = calculate_comprehensive_metrics(test_df_model, "Test")
                metrics['Model'] = model
                comparison_results.append(metrics)
        except Exception as e:
            print(f"⚠️ Could not load {model}: {e}")
    
    if comparison_results:
        comparison_df = pd.DataFrame(comparison_results)
        comparison_df = comparison_df[['Model', 'MAE', 'RMSE', 'MAPE (%)', 'R²', 'Median AE', 'Max Error']]
        comparison_df = comparison_df.sort_values('MAE')
        
        print(f"\n🏆 MODEL COMPARISON SUMMARY")
        print("=" * 80)
        print(comparison_df.to_string(index=False, float_format='%.4f'))
        
        # Create comparison visualization
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle('Model Comparison', fontsize=16, fontweight='bold')
        
        # MAE comparison
        ax1 = axes[0]
        bars1 = ax1.bar(comparison_df['Model'], comparison_df['MAE'], color='skyblue', edgecolor='black')
        ax1.set_title('Mean Absolute Error (MAE)')
        ax1.set_ylabel('MAE')
        ax1.tick_params(axis='x', rotation=45)
        ax1.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar in bars1:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{height:.2f}', ha='center', va='bottom')
        
        # R² comparison
        ax2 = axes[1]
        bars2 = ax2.bar(comparison_df['Model'], comparison_df['R²'], color='lightgreen', edgecolor='black')
        ax2.set_title('R-squared (R²)')
        ax2.set_ylabel('R²')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar in bars2:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{height:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        return comparison_df
    
    return None

# Run model comparison
if len(available_models) > 1:
    print("🔄 Loading all models for comparison...")
    comparison_results = compare_all_models()
else:
    print("ℹ️ Only one model available for analysis")

## 10. Export Results

In [ ]:
# Create a summary report
if test_df is not None:
    print(f"\n📄 ANALYSIS SUMMARY FOR {MODEL_NAME.upper()}")
    print("=" * 60)
    print(f"📊 Dataset: {len(test_df):,} test samples")
    if 'timestamp' in test_df.columns:
        print(f"📅 Time period: {test_df['timestamp'].min()} to {test_df['timestamp'].max()}")
    
    print(f"\n🎯 Key Performance Metrics:")
    print(f"  • MAE (Mean Absolute Error): {test_metrics['MAE']:.4f}")
    print(f"  • RMSE (Root Mean Square Error): {test_metrics['RMSE']:.4f}")
    print(f"  • MAPE (Mean Absolute Percentage Error): {test_metrics['MAPE (%)']:.2f}%")
    print(f"  • R² (Coefficient of Determination): {test_metrics['R²']:.4f}")
    
    # Calculate additional insights
    residuals = test_df['true_count'] - test_df['pred_count']
    print(f"\n📈 Additional Insights:")
    print(f"  • Mean residual (bias): {np.mean(residuals):.4f}")
    print(f"  • Predictions within 10% of actual: {(np.abs(residuals/test_df['true_count']) <= 0.1).mean()*100:.1f}%")
    print(f"  • Predictions within 20% of actual: {(np.abs(residuals/test_df['true_count']) <= 0.2).mean()*100:.1f}%")
    
    # Export detailed results
    export_df = test_df.copy()
    export_df['residual'] = residuals
    export_df['abs_error'] = np.abs(residuals)
    export_df['pct_error'] = (residuals / test_df['true_count']) * 100
    
    export_filename = f"{MODEL_NAME}_detailed_analysis.csv"
    export_df.to_csv(export_filename, index=False)
    print(f"\n💾 Detailed analysis exported to: {export_filename}")
    
    print(f"\n✅ Analysis complete for {MODEL_NAME.upper()}!")